In [1]:
TEST_DATA = "test_set_clean.dat"
TRAIN_DATA = "train_set_clean.dat"
TEST_LABEL_FILE = "labels_test_set_clean.txt"

MAXLINES = None # int(4e5)
background_label = "0"

In [2]:
test_sequences = list()

print("Sequences")
for i, line in enumerate(open(TEST_DATA, "rt")):
    if i==0:
        alphabet_size = int(line.split()[-1])
        continue
    elif MAXLINES and i+1 >= MAXLINES + 1 or len(line.strip()) == 0: # +1 on maxlines for the header
        continue
    line = line.split()
    test_sequences.append( list( int(x) for x in line[2:]) ) 
    if i % int(10e5) == 0:
        print("Line: ", i)

test_labels = list()
print("Labels")
for i, line in enumerate(open(TEST_LABEL_FILE, "rt")):
    if MAXLINES and i+1 >= MAXLINES or len(line.strip()) == 0:
        continue
    if line.split()[-1] != background_label:
        test_labels.append(1)
    else:
        test_labels.append(0)
assert(len(test_sequences) == len(test_labels))
print("Got {} labeled sequences".format(len(test_sequences)))

n_test_sequences = len(test_sequences)

Sequences
Line:  1000000
Line:  2000000
Line:  3000000
Line:  4000000
Line:  5000000
Line:  6000000
Line:  7000000
Labels
Got 7686952 labeled sequences


In [3]:
benign_lines = dict()
malign_lines = dict()

def do_multimap(d: dict, s: str):
    """
    Counts the strings in the relevant maps.
    """
    if s in d:
        d[s] += 1
    else:
        d[s] = 1

for seq, label in zip(test_sequences, test_labels):
    seq_str = " ".join(str(x) for x in seq)
    if label == 0:
        do_multimap(benign_lines, seq_str)
    elif label == 1:
         do_multimap(malign_lines, seq_str)
            
print(len(benign_lines), len(malign_lines))

1252793 2635


In [4]:
malign_counts = list()
benign_counts = list()

overlapping_strings = set(benign_lines.keys()).intersection(malign_lines.keys())
for overlap in overlapping_strings:
    #print("Benign counts: {}, malign counts: {}, len: {}".format(benign_lines[overlap], malign_lines[overlap], \
    #                                                             len(overlap.split())))
    malign_counts.append((malign_lines[overlap], benign_lines[overlap], len(overlap.split()), overlap))
    benign_counts.append((benign_lines[overlap], malign_lines[overlap], len(overlap.split())))
malign_counts.sort(reverse=True)
benign_counts.sort(reverse=True)
malign_counts[:10], benign_counts[:10]

([(214383,
   2395,
   10,
   '487601 487601 487601 487601 487601 487601 487601 487601 487601 487601'),
  (106357,
   175,
   10,
   '487564 487564 487564 487564 487564 487564 487564 487564 487564 487564'),
  (42503,
   151,
   10,
   '487565 487565 487565 487565 487565 487565 487565 487565 487565 487565'),
  (27566,
   6571,
   10,
   '487602 487602 487602 487602 487602 487602 487602 487602 487602 487602'),
  (20165,
   305,
   10,
   '113133 113133 113133 113133 113133 113133 113133 113133 113133 113133'),
  (19076,
   1027,
   10,
   '440908 440908 440908 440908 440908 440908 440908 440908 440908 440908'),
  (17107,
   632,
   10,
   '394252 394252 394252 394252 394252 394252 394252 394252 394252 394252'),
  (13026,
   1096,
   10,
   '394253 394253 394253 394253 394253 394253 394253 394253 394253 394253'),
  (11493,
   98,
   10,
   '113170 113170 113170 113170 113170 113170 113170 113170 113170 113170'),
  (9070,
   57,
   10,
   '113134 113134 113134 113134 113134 113134 113134 1

In [5]:
most_costly_strings = set()
for i in range(10):
    most_costly_strings.add(malign_counts[i][-1])
most_costly_strings

{'113133 113133 113133 113133 113133 113133 113133 113133 113133 113133',
 '113134 113134 113134 113134 113134 113134 113134 113134 113134 113134',
 '113170 113170 113170 113170 113170 113170 113170 113170 113170 113170',
 '394252 394252 394252 394252 394252 394252 394252 394252 394252 394252',
 '394253 394253 394253 394253 394253 394253 394253 394253 394253 394253',
 '440908 440908 440908 440908 440908 440908 440908 440908 440908 440908',
 '487564 487564 487564 487564 487564 487564 487564 487564 487564 487564',
 '487565 487565 487565 487565 487565 487565 487565 487565 487565 487565',
 '487601 487601 487601 487601 487601 487601 487601 487601 487601 487601',
 '487602 487602 487602 487602 487602 487602 487602 487602 487602 487602'}

In [6]:
counts = list()
for mal_string, mal_count in malign_lines.items():
    if mal_string in overlapping_strings:
        continue
    counts.append(mal_count)
counts.sort(reverse=True)
print(counts[:10])

[135186, 10700, 3968, 1865, 1620, 1620, 1138, 939, 900, 837]


In [7]:
train_counter = dict()

for i, line in enumerate(open(TRAIN_DATA, "rt")):
    if i == 0:
        continue
    elif i%int(1e6) == 0:
        print("Line ", i)
    line = " ".join(line.split()[2:])
    if line in malign_lines:
        do_multimap(train_counter, line)
print("Total amount of sequences: ", i-1)

n_train_sequences = i-1

Line  1000000
Line  2000000
Line  3000000
Line  4000000
Line  5000000
Line  6000000
Line  7000000
Line  8000000
Line  9000000
Line  10000000
Line  11000000
Line  12000000
Line  13000000
Line  14000000
Line  15000000
Line  16000000
Line  17000000
Line  18000000
Line  19000000
Line  20000000
Line  21000000
Line  22000000
Line  23000000
Line  24000000
Line  25000000
Line  26000000
Line  27000000
Line  28000000
Line  29000000
Line  30000000
Line  31000000
Line  32000000
Line  33000000
Line  34000000
Line  35000000
Total amount of sequences:  35422725


In [8]:
for seq, count in train_counter.items():
    print("Train data: {}, test data: {}".format(count, malign_lines[seq]))

Train data: 91385, test data: 49
Train data: 125183, test data: 1450
Train data: 160, test data: 1
Train data: 461, test data: 66
Train data: 45, test data: 1
Train data: 591, test data: 5
Train data: 674, test data: 5
Train data: 780, test data: 4
Train data: 1043, test data: 4
Train data: 1337, test data: 2
Train data: 2898, test data: 2
Train data: 311377, test data: 190
Train data: 1201, test data: 1
Train data: 357, test data: 1
Train data: 286, test data: 1
Train data: 227, test data: 1
Train data: 233, test data: 1
Train data: 271, test data: 1
Train data: 364, test data: 1
Train data: 579, test data: 1
Train data: 155792, test data: 152
Train data: 1332, test data: 4
Train data: 624, test data: 5
Train data: 585, test data: 5
Train data: 2493, test data: 4
Train data: 1433, test data: 4
Train data: 1317, test data: 6
Train data: 1293, test data: 6
Train data: 1381, test data: 7
Train data: 1529, test data: 7
Train data: 1817, test data: 4
Train data: 2119, test data: 4
Train da

In [9]:
print(len(train_counter), len(malign_lines))

987 2635


In [10]:
malign_counts.sort(reverse=True)
for count, _, _, line in malign_counts:
    if line in train_counter:
        if line in benign_lines:
            print("Malign test set: {}, p: {}, benign test set: {}, p: {}, train set: {}, p: {}".format(count, count / n_test_sequences, benign_lines[line], benign_lines[line] / n_test_sequences,  train_counter[line], train_counter[line] / n_train_sequences))
        else:
            print("Malign test set: {}, p: {}, benign test set: {}, p: {}, train set: {}, p: {}".format(count, count / n_test_sequences, 0, 0, train_counter[line], train_counter[line] / n_train_sequences))
    else:
        if line in benign_lines:
            print("Malign test set: {}, p: {}, benign test set: {}, p: {}, train set: {}, p: {}".format(count, count / n_test_sequences, benign_lines[line], benign_lines[line] / n_test_sequences,  0, 0))
        else:
            print("Malign test set: {}, p: {}, benign test set: {}, p: {}, train set: {}, p: {}".format(count, count / n_test_sequences, 0, 0, 0, 0))


Malign test set: 214383, p: 0.027889207581886812, benign test set: 2395, p: 0.0003115669253561099, train set: 11338, p: 0.0003200770127086496
Malign test set: 106357, p: 0.013836043206722248, benign test set: 175, p: 2.276585049574916e-05, train set: 2010, p: 5.674323474549177e-05
Malign test set: 42503, p: 0.005529239677833295, benign test set: 151, p: 1.964367671347499e-05, train set: 838, p: 2.365712970981199e-05
Malign test set: 27566, p: 0.0035860767700904078, benign test set: 6571, p: 0.0008548251634718156, train set: 29560, p: 0.0008344925468043466
Malign test set: 20165, p: 0.0026232764299816106, benign test set: 305, p: 3.967762514973425e-05, train set: 116, p: 3.274733945510968e-06
Malign test set: 19076, p: 0.00248160779461092, benign test set: 1027, p: 0.00013360301976648222, train set: 283, p: 7.989221608444861e-06
Malign test set: 17107, p: 0.0022254594538901767, benign test set: 632, p: 8.221724293321982e-05, train set: 1556, p: 4.3926603613922984e-05
Malign test set: 13

In [11]:
n_train_sequences, 4704

(35422725, 4704)